# Extração

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
import os
import pymysql
from dotenv import load_dotenv
from collections import Counter

data_path = '../data/raw/'

csv_files = [
    'campanhas_marketing.csv',
    'clientes.csv',
    'engajamento_redes_sociais.csv',
    'lojas.csv',
    'produtos.csv',
    'promocoes.csv',
    'vendas_atualizado.csv'
]

json_file = 'vendas_online (1).json'

dataframes = {}

for file in csv_files:
    try:
        file_path = os.path.join(data_path, file)
        df_name = file.replace('.csv', '')
        dataframes[df_name] = pd.read_csv(file_path)
        print(f"Arquivo {file} lido com sucesso. DataFrame criado: {df_name}")
        print(f"Primeiras 5 linhas do {df_name}:")
        print(dataframes[df_name].head())
    except FileNotFoundError:
        print(f"Erro: Arquivo {file} não encontrado no caminho especificado: {file_path}")
    except Exception as e:
        print(f"Ocorreu um erro ao ler o arquivo {file}: {e}")

print("\nExtraindo arquivo JSON")
try:
    json_file_path = os.path.join(data_path, json_file)
    df_name = json_file.replace('.json', '').replace(' (1)', '')
    dataframes[df_name] = pd.read_json(json_file_path)
    print(f"Arquivo {json_file} lido com sucesso. DataFrame criado: {df_name}")
    print(f"Primeiras 5 linhas do {df_name}:")
    print(dataframes[df_name].head())
except FileNotFoundError:
    print(f"Erro: Arquivo {json_file} não encontrado no caminho especificado: {json_file_path}")
except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo {json_file}: {e}")

# Panorama Geral

## Verificando tipos, valores ausentes e linhas/colunas duplicadas

In [2]:
for df_name, df in dataframes.items():
    print(f"\nVerificando DataFrame: {df_name}")

    print(f"Informações gerais do DataFrame {df_name}:")
    df.info()

    print(f"Quantidade de valores ausentes por coluna no DataFrame {df_name}:")
    print(df.isnull().sum())

    print(f"Quantidade de linhas duplicadas no DataFrame {df_name}:")
    num_duplicates = df.duplicated().sum()
    print(num_duplicates)
    if num_duplicates > 0:
        print(f"Existem {num_duplicates} linhas duplicadas no DataFrame {df_name}.")

    print("-" * 70)


Verificando DataFrame: campanhas_marketing
Informações gerais do DataFrame campanhas_marketing:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID_Campanha       500 non-null    int64 
 1   Nome_Campanha     500 non-null    object
 2   Canal_Divulgacao  500 non-null    object
 3   Publico_Alvo      500 non-null    object
 4   Data_Inicio       500 non-null    object
 5   Data_Fim          500 non-null    object
dtypes: int64(1), object(5)
memory usage: 23.6+ KB
Quantidade de valores ausentes por coluna no DataFrame campanhas_marketing:
ID_Campanha         0
Nome_Campanha       0
Canal_Divulgacao    0
Publico_Alvo        0
Data_Inicio         0
Data_Fim            0
dtype: int64
Quantidade de linhas duplicadas no DataFrame campanhas_marketing:
0
----------------------------------------------------------------------

Verificando DataFra

# Transformação

### Corrigindo os tipos de dados dos arquivos CSV

In [3]:
if 'campanhas_marketing' in dataframes:
    df_campanhas = dataframes['campanhas_marketing']
    print("\nTratando DataFrame: campanhas_marketing")
    try:
        df_campanhas['Data_Inicio'] = pd.to_datetime(df_campanhas['Data_Inicio'])
        df_campanhas['Data_Fim'] = pd.to_datetime(df_campanhas['Data_Fim'])
        print("Colunas de data em 'campanhas_marketing' convertidas para datetime.")
    except KeyError as e:
        print(f"Erro na limpeza de 'campanhas_marketings': Coluna {e} não encontrada.")
    except Exception as e:
        print(f"Ocorreu um erro inesperado na limpeza de 'campanhas_marketings': {e}")
else:
    print("\nAviso: DataFrame 'campanhas_marketing' não encontrado para limpeza.")
print("-" * 30)

if 'clientes' in dataframes:
    print("\nVerificando DataFrame: clientes.")
else:
     print("\nAviso: DataFrame 'clientes' não encontrado para verificação.")
print("-" * 30)

if 'engajamento_redes_sociais' in dataframes:
    df_engajamento = dataframes['engajamento_redes_sociais']
    print("\nTratando DataFrame: engajamento_redes_sociais")
    try:
        df_engajamento['Data_Engajamento'] = pd.to_datetime(df_engajamento['Data_Engajamento'])
        print("Coluna de data em 'engajamento_redes_sociais' convertida para datetime.")
    except KeyError as e:
        print(f"Erro na limpeza de 'engajamento_redes': Coluna {e} não encontrada.")
    except Exception as e:
        print(f"Ocorreu um erro inesperado na limpeza de 'engajamento_redes': {e}")
else:
     print("\nDataFrame 'engajamento_redes' não encontrado para limpeza.")
print("-" * 30)

if 'lojas' in dataframes:
    print("\nVerificando DataFrame: lojas")
else:
     print("\nDataFrame 'lojas' não encontrado para verificação.")
print("-" * 30)

if 'produtos' in dataframes:
    df_produtos = dataframes['produtos']
    print("\nTratando DataFrame: produtos (Limpando nomes de produtos)")
    try:
        if 'Nome_Produto' in df_produtos.columns:
            print("Coluna 'Nome_Produto' encontrada. Aplicando limpeza com Regex...")

            pattern_regex = r'\s+\d+$'
            df_produtos['Nome_Produto'] = df_produtos['Nome_Produto'].str.replace(pattern_regex, '', regex=True).str.strip()

            print("Coluna 'Nome_Produto' limpa no DataFrame.")
            print(df_produtos.head())

            dataframes['produtos'] = df_produtos
            print("DataFrame 'produtos' atualizado no dicionário.")

        else:
            print("Erro na limpeza de 'produtos': Coluna 'Nome_Produto' não encontrada.")
    except Exception as e:
        print(f"Ocorreu um erro inesperado na limpeza de 'produtos': {e}")
else:
    print("\nAviso: DataFrame 'produtos' não encontrado para limpeza.")
print("-" * 30)

if 'promocoes' in dataframes:
    df_promocoes = dataframes['promocoes']
    print("\nTratando DataFrame: promocoes")
    try:
        df_promocoes['Data_Inicio'] = pd.to_datetime(df_promocoes['Data_Inicio'])
        df_promocoes['Data_Fim'] = pd.to_datetime(df_promocoes['Data_Fim'])
        print("Colunas de data em 'promocoes' convertidas para datetime")
    except KeyError as e:
        print(f"Erro na limpeza de 'promocoes': Coluna {e} não encontrada")
    except Exception as e:
        print(f"Ocorreu um erro inesperado na limpeza de 'promocoes': {e}")
else:
     print("\nAviso: DataFrame 'promocoes' não encontrado para limpeza.")
print("-" * 30)

if 'vendas_atualizado' in dataframes:
    df_vendas_atualizado = dataframes['vendas_atualizado']
    print("\nTratando DataFrame: vendas_atualizado")
    try:
        df_vendas_atualizado['Data_Venda'] = pd.to_datetime(df_vendas_atualizado['Data_Venda'])
        print("Coluna de timestamp em 'vendas_atualizado' convertida para datetime.")
    except KeyError as e:
         print(f"Erro na limpeza de 'vendas_atualizado': Coluna {e} não encontrada.")
    except Exception as e:
        print(f"Ocorreu um erro inesperado na limpeza de 'vendas_atualizado': {e}")
else:
     print("\nAviso: DataFrame 'vendas_atualizado' não encontrado para limpeza.")
print("-" * 30)

if 'vendas_online' in dataframes:
    print("\nVerificando DataFrame: vendas_online")
else:
     print("\nAviso: DataFrame 'vendas_online' não encontrado no dicionário.")
print("-" * 30)


Tratando DataFrame: campanhas_marketing
Colunas de data em 'campanhas_marketing' convertidas para datetime.
------------------------------

Verificando DataFrame: clientes.
------------------------------

Tratando DataFrame: engajamento_redes_sociais
Coluna de data em 'engajamento_redes_sociais' convertida para datetime.
------------------------------

Verificando DataFrame: lojas
------------------------------

Tratando DataFrame: produtos (Limpando nomes de produtos)
Coluna 'Nome_Produto' encontrada. Aplicando limpeza com Regex...
Coluna 'Nome_Produto' limpa no DataFrame.
   ID_Produto Categoria_Produto  Preco_Base     Nome_Produto
0           1       Eletrônicos  148.010048           Câmera
1           2     Casa e Jardim  232.662699          Lâmpada
2           3          Esportes  386.939967            Skate
3           4            Livros  375.004266         Infantil
4           5          Esportes  415.953038  Bola de futebol
DataFrame 'produtos' atualizado no dicionário.
-----

### Verificando valores ausentes e colunas duplicadas do arquivo JSON

In [4]:
df_vendas_online = dataframes['vendas_online']

print("\nInformações gerais do DataFrame vendas_online:")
df_vendas_online.info()
print("-" * 30)

print("\nQuantidade de valores ausentes por coluna:")
print(df_vendas_online.isnull().sum())
print("-" * 30)

print("\nNomes das colunas no DataFrame:")
print(df_vendas_online.columns.tolist())
print("-" * 30)

column_names = df_vendas_online.columns.tolist()
lower_case_names = [name.lower() for name in column_names]

print("Verificando por nomes de colunas duplicados:")
if len(lower_case_names) != len(set(lower_case_names)):
    print("Nomes de colunas duplicados encontrados:")

    name_counts = Counter(lower_case_names)
    for name, count in name_counts.items():
        if count > 1:
            original_names = [col for col in column_names if col.lower() == name]
            print(f"  '{name}' aparece {count} vezes com as grafias: {original_names}")
else:
    print("Não foram encontrados nomes de colunas duplicados.")


Informações gerais do DataFrame vendas_online:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271781 entries, 0 to 271780
Data columns (total 26 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   idVenda          20000 non-null   float64
 1   idProduto        20000 non-null   float64
 2   idLoja           20000 non-null   float64
 3   dataVenda        20000 non-null   object 
 4   qtd              20000 non-null   float64
 5   precoUnit        20000 non-null   float64
 6   idCliente        20000 non-null   float64
 7   desconto         20000 non-null   float64
 8   promocao         20000 non-null   float64
 9   estoqueInicial   20000 non-null   float64
 10  estoqueFinal     20000 non-null   float64
 11  devolucao        20000 non-null   float64
 12  canalVenda       20000 non-null   object 
 13  ID_Venda         251781 non-null  float64
 14  ID_Produto       251781 non-null  float64
 15  ID_Loja          251781 non-null  flo

### Consolidando as colunas duplicadas e corrigindo os tipos de dados do arquivo JSON

In [5]:
df_vendas_online = dataframes['vendas_online']

column_mapping = {
    'idVenda': 'ID_Venda',
    'idProduto': 'ID_Produto',
    'idLoja': 'ID_Loja',
    'dataVenda': 'Data_Venda',
    'qtd': 'Quantidade',
    'precoUnit': 'Preco_Unitario',
    'idCliente': 'ID_Cliente',
    'desconto': 'Desconto',
    'promocao': 'Promocao',
    'estoqueInicial': 'Estoque_Inicial',
    'estoqueFinal': 'Estoque_Final',
    'devolucao': 'Devolucao',
    'canalVenda': 'Canal_Venda'
}

print("Consolidando colunas:")
try:
    for lower_col, upper_col in column_mapping.items():
        if lower_col in df_vendas_online.columns and upper_col in df_vendas_online.columns:
            df_vendas_online[lower_col] = df_vendas_online[lower_col].fillna(df_vendas_online[upper_col])
            print(f"Consolidado: '{upper_col}' em '{lower_col}'.")
        else:
            print(f"Ambas as colunas '{lower_col}' e '{upper_col}' não encontradas.")

    data_column_name = 'dataVenda'
    if data_column_name in df_vendas_online.columns:
         df_vendas_online[data_column_name] = pd.to_datetime(df_vendas_online[data_column_name])
         print(f"Coluna '{data_column_name}' convertida para datetime.")
    else:
        print(f"Coluna de data '{data_column_name}' não encontrada para conversão.")

    columns_to_drop = list(column_mapping.values())

    existing_columns_to_drop = [col for col in columns_to_drop if col in df_vendas_online.columns]

    if existing_columns_to_drop:
        df_vendas_online = df_vendas_online.drop(columns=existing_columns_to_drop)
        print("Colunas redundantes removidas:", existing_columns_to_drop)
    else:
        print("Nenhuma coluna redundante encontrada para remover com base no mapeamento.")

except Exception as e:
    print(f"Ocorreu um erro durante a consolidação ou limpeza: {e}")

print("\nVerificando DataFrame vendas_online após consolidação e limpeza:")
df_vendas_online.info()
print("-" * 30)
print("Quantidade de valores ausentes após consolidação:")
print(df_vendas_online.isnull().sum())

dataframes['vendas_online'] = df_vendas_online

Consolidando colunas:
Consolidado: 'ID_Venda' em 'idVenda'.
Consolidado: 'ID_Produto' em 'idProduto'.
Consolidado: 'ID_Loja' em 'idLoja'.
Consolidado: 'Data_Venda' em 'dataVenda'.
Consolidado: 'Quantidade' em 'qtd'.
Consolidado: 'Preco_Unitario' em 'precoUnit'.
Consolidado: 'ID_Cliente' em 'idCliente'.
Consolidado: 'Desconto' em 'desconto'.
Consolidado: 'Promocao' em 'promocao'.
Consolidado: 'Estoque_Inicial' em 'estoqueInicial'.
Consolidado: 'Estoque_Final' em 'estoqueFinal'.
Consolidado: 'Devolucao' em 'devolucao'.
Consolidado: 'Canal_Venda' em 'canalVenda'.
Coluna 'dataVenda' convertida para datetime.
Colunas redundantes removidas: ['ID_Venda', 'ID_Produto', 'ID_Loja', 'Data_Venda', 'Quantidade', 'Preco_Unitario', 'ID_Cliente', 'Desconto', 'Promocao', 'Estoque_Inicial', 'Estoque_Final', 'Devolucao', 'Canal_Venda']

Verificando DataFrame vendas_online após consolidação e limpeza:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271781 entries, 0 to 271780
Data columns (total 13 colu

In [6]:
df_vendas_online = dataframes['vendas_online']

integer_columns = [
    'idVenda',
    'idProduto',
    'idLoja',
    'qtd',
    'idCliente',
    'promocao',
    'estoqueInicial',
    'estoqueFinal',
    'devolucao'
]

try:
    for col in integer_columns:
        if col in df_vendas_online.columns and df_vendas_online[col].dtype == 'float64':
            df_vendas_online[col] = df_vendas_online[col].astype(int)
            print(f"Coluna '{col}' convertida para int")
        elif col not in df_vendas_online.columns:
             print(f"Coluna '{col}' não encontrada no DataFrame")
        else:
             print(f"Coluna '{col}' não é float64 ({df_vendas_online[col].dtype})")

except Exception as e:
    print(f"Ocorreu um erro durante a conversão para inteiro: {e}")

print("\nVerificando DataFrame vendas_online após conversão de inteiros:")
df_vendas_online.info()
print("-" * 30)
print("Quantidade de valores ausentes após conversão:")
print(df_vendas_online.isnull().sum())
print("=" * 50)

dataframes['vendas_online'] = df_vendas_online

Coluna 'idVenda' convertida para int
Coluna 'idProduto' convertida para int
Coluna 'idLoja' convertida para int
Coluna 'qtd' convertida para int
Coluna 'idCliente' convertida para int
Coluna 'promocao' convertida para int
Coluna 'estoqueInicial' convertida para int
Coluna 'estoqueFinal' convertida para int
Coluna 'devolucao' convertida para int

Verificando DataFrame vendas_online após conversão de inteiros:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271781 entries, 0 to 271780
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   idVenda         271781 non-null  int32         
 1   idProduto       271781 non-null  int32         
 2   idLoja          271781 non-null  int32         
 3   dataVenda       271781 non-null  datetime64[ns]
 4   qtd             271781 non-null  int32         
 5   precoUnit       271781 non-null  float64       
 6   idCliente       271781 non-null  int32         


### Salvando o novo dataframe tratado

In [ ]:
df_vendas_online_consolidado = dataframes['vendas_online']
processed_data_path = '../data/processed/'
output_json_file = 'vendas_online_consolidado.json'
output_json_path = os.path.join(processed_data_path, output_json_file)

os.makedirs(processed_data_path, exist_ok=True)

try:
    df_vendas_online_consolidado.to_json(output_json_path, orient='records', indent=4, date_format='iso')
    print(f"DataFrame 'vendas_online' salvo com sucesso em: {output_json_path}")

except Exception as e:
    print(f"Ocorreu um erro ao salvar o arquivo JSON: {e}")

## Conexão com o banco de dados (Carregamento)

### Testando conexão

In [ ]:
pymysql.install_as_MySQLdb()

load_dotenv()
MYSQL_HOST = os.getenv('MYSQL_HOST')
MYSQL_DB = os.getenv('MYSQL_DB')
MYSQL_USER = os.getenv('MYSQL_USER')
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')

DATABASE_URI = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}/{MYSQL_DB}'

engine = None
try:
    print(f"Tentando conectar ao banco de dados")
    engine = create_engine(DATABASE_URI)

    with engine.connect() as connection:
        print(f"\nConexão com o banco de dados '{MYSQL_DB}' estabelecida com sucesso!")

except SQLAlchemyError as e:
    print(f"\nErro ao conectar com o banco de dados: {e}")
    print(f"Tipo de erro: {type(e)}")
except Exception as e:
    print(f"\nOcorreu um erro inesperado ao criar o motor de conexão: {e}")

if 'dataframes' not in globals() or not isinstance(dataframes, dict):
     print("\nO dicionário 'dataframes' não foi encontrado ou não é um dicionário.")
else:
    if engine is not None:
        print("\n--- Carregando DataFrame 'lojas' para o MySQL ---")
        try:
            if 'lojas' in dataframes:
                 df_lojas = dataframes['lojas']
                 df_lojas.to_sql(
                     name='lojas', con=engine, if_exists='replace', index=False
                 )

                 print("DataFrame 'lojas' carregado com sucesso para a tabela 'lojas' no MySQL.")
            else:
                 print("DataFrame 'lojas' não encontrado no dicionário 'dataframes'.")

        except SQLAlchemyError as e:
            print(f"Erro ao carregar o DataFrame 'lojas' para o banco de dados: {e}")
            if hasattr(e, 'orig') and hasattr(e.orig, 'args'):
                print(f"Detalhes do erro MySQL: {e.orig.args}")
        except Exception as e:
            print(f"Ocorreu um erro inesperado ao carregar o DataFrame 'lojas': {e}")

    else:
        print("\nNão foi possível carregar o DataFrame 'lojas' pois a conexão com o banco de dados falhou.")

### Conectando o resto

In [ ]:
dataframes_to_load = [
    'campanhas_marketing',
    'clientes',
    'engajamento_redes_sociais',
    'produtos',
    'promocoes',
    'vendas_atualizado',
    'vendas_online'
]

if 'engine' in locals() and engine is not None:
    print("Motor de conexão com o banco de dados disponível.")

    for df_name in dataframes_to_load:
        print(f"\nTentando carregar DataFrame '{df_name}' para a tabela '{df_name}'...")
        try:
            if df_name in dataframes:
                df_to_load = dataframes[df_name]
                df_to_load.to_sql(
                    name=df_name, con=engine, if_exists='replace', index=False
                )
                print(f"DataFrame '{df_name}' carregado com sucesso para a tabela '{df_name}'")

            else:
                print(f"DataFrame '{df_name}' não encontrado no dicionário 'dataframes'")

        except SQLAlchemyError as e:
            print(f"Erro ao carregar o DataFrame '{df_name}' para o banco de dados: {e}")
            if hasattr(e, 'orig') and hasattr(e.orig, 'args'):
                print(f"Detalhes do erro MySQL: {e.orig.args}")
        except Exception as e:
            print(f"Ocorreu um erro inesperado ao carregar o DataFrame '{df_name}': {e}")

else:
    print("\nErro: O motor de conexão com o banco de dados 'engine' não foi criado. Verifique a célula de conexão anterior")

# Explorando o banco de dados SQLite (marketing.db)

In [10]:
sqlite_db_path = '../data/raw/marketing.db'

sqlite_uri = f'sqlite:///{sqlite_db_path}'

sqlite_engine = None
try:
    print(f"Tentando conectar ao banco de dados SQLite: {sqlite_db_path}")
    sqlite_engine = create_engine(sqlite_uri)

    with sqlite_engine.connect() as connection:
        print("Conexão com o banco de dados SQLite estabelecida com sucesso!")

        inspector = sqlalchemy.inspect(sqlite_engine)
        sqlite_tables = inspector.get_table_names()

        print("\nTabelas encontradas no banco de dados SQLite:")
        if sqlite_tables:
            for table_name in sqlite_tables:
                print(f"- {table_name}")
        else:
            print("Nenhuma tabela encontrada")

except SQLAlchemyError as e:
    print(f"\nErro ao conectar ou explorar o banco de dados SQLite: {e}")
except Exception as e:
    print(f"\nOcorreu um erro inesperado: {e}")

Tentando conectar ao banco de dados SQLite: ../data/raw/marketing.db
Conexão com o banco de dados SQLite estabelecida com sucesso!

Tabelas encontradas no banco de dados SQLite:
- campanhas_marketing
- engajamento_redes_sociais
- promocoes


### Lendo tabelas do banco de dados SQLite

In [ ]:
if 'sqlite_engine' in locals() and sqlite_engine is not None:
    sqlite_tables_to_read = [
        'campanhas_marketing',
        'engajamento_redes_sociais',
        'promocoes'
    ]

    for table_name in sqlite_tables_to_read:
        print(f"\nLendo tabela '{table_name}' do SQLite")
        try:
            df_sqlite = pd.read_sql_table(
                table_name=table_name, con=sqlite_engine
            )

            df_name_in_dict = f"{table_name}_sqlite"
            dataframes[df_name_in_dict] = df_sqlite
            print(f"Tabela '{table_name}' lida com sucesso. DataFrame criado: '{df_name_in_dict}'.")

            print(f"Informações do DataFrame '{df_name_in_dict}':")
            df_sqlite.info()
            print(f"\nPrimeiras 5 linhas do DataFrame '{df_name_in_dict}':")
            print(df_sqlite.head())
            print("-" * 30)

        except Exception as e:
            print(f"Ocorreu um erro ao ler a tabela '{table_name}' do SQLite: {e}")

else:
    print("\nErro: O motor de conexão SQLite 'sqlite_engine' não foi criado.")

# Salvando o resto dos DataFrames tratatos

In [ ]:
processed_data_path = '../data/processed/'

os.makedirs(processed_data_path, exist_ok=True)
dataframes_to_save = dataframes

for df_name, df in dataframes_to_save.items():
    output_csv_file = f'{df_name}_processed.csv'
    output_csv_path = os.path.join(processed_data_path, output_csv_file)

    print(f"\nSalvando DataFrame '{df_name}' para '{output_csv_file}'")

    try:
        df.to_csv(output_csv_path, index=False)
        print(f"DataFrame '{df_name}' salvo com sucesso.")

    except Exception as e:
        print(f"Ocorreu um erro ao salvar o DataFrame '{df_name}': {e}")